In [ ]:
using LinearAlgebra
using Plots
using Ripserer
using Distances
using Plots
using DataFrames, CSV
using DelimitedFiles
using AbstractAlgebra
using MultivariateStats #for pca
using IterativeSolvers
using NearestNeighbors
include("mv_function_bens_alg.jl")


In [ ]:
#Globals
KKK = 2
K = 20 # in KNN
OG_K = 10
ITER = 10    # iterations of algorithm
break_index = KKK
min_loss = 1000
input_file ="./cool_big_data.csv"
output_file ="./point_color"

a_ = []
b_ = []
c_ = []
d_ = []
loss_ = []

In [ ]:
# Loss function 0.5 instead of pi

function loss(points, coord_ab, coord_cd)
    a = mod(coord_cd[points[1]],1.0)
    b = mod(coord_cd[points[2]],1.0)
    c = mod(coord_ab[points[3]],1.0)
    d = mod(coord_ab[points[4]],1.0)

    """
    temp_1 = mod(a-b, 1.0)
    if temp_1 < 0
        print("ERROR")
        #break
    end
    if temp_1 < 0.5
        temp_1 -= 0.5
    end

    temp_2 = mod(c-d, 1.0)
    if temp_1 < 0.5
        temp_2 -= 0.5
    end
    #println("temp_1 = ", temp_1, " temp_2 = ", temp_2)
    """
    dif_1 = 0.5 - mod(a-b, 1.0)

    dif_2 = 0.5 - mod(c-d, 1.0)

    loss = (dif_1)^2 +  (dif_2)^2

    return loss
end

In [ ]:
# Load Datab
matrix = readdlm(input_file, ',', Float64)
data = []
for row in 1:length(matrix[:,1])
    arr = [matrix[row,i] for i in 1:length(matrix[1,:])]
    push!(data, arr)
end

# Do zeroth iteration of algerithm with random points

# Random distinct start points,
# Could be issues with distinct points yielding same piercing hole, and therefore cant do MV
a_0,b_0,c_0,d_0 = rand(1:25), rand(25:50), rand(50:75), rand(75:100)


# Store to see progress 

push!(a_, a_0)
push!(b_, b_0)
push!(c_, c_0)
push!(d_, d_0)


data_matrix = zeros(Float64, (3, length(data)))

for col in 1:length(data)
    data_matrix[:,col] = [data[col][1],data[col][2],data[col][3]]
end

kdtree = KDTree(data_matrix)


In [ ]:
"""
# Perfect start
vec_0 = Array([1,0,0])
vec_1 = Array([0,1,0])
vec_2 = Array([0,0,1])

a_0_set, dists = knn(kdtree, vec_0,K, true)
c_0_set, dists = knn(kdtree, vec_1,K, true)

vec_0 = Array([-1,0,0])
vec_1 = Array([0,-1,0])
vec_2 = Array([0,0,-1])

b_0_set, dists = knn(kdtree, vec_0,K, true)
d_0_set, dists = knn(kdtree, vec_1,K, true)


push!(a_, a_0_set[1])
push!(b_, b_0_set[1])
push!(c_, c_0_set[1])
push!(d_, d_0_set[1])
"""


In [ ]:

for k in 1:ITER
    println("Current points: ")
    l = length(a_)
    println("[" * string(a_[l]-1) * "," * string(b_[l]-1) * "," * string(c_[l]-1) * "," * string(d_[l]-1) * "]")
    points = [a_[l], b_[l], c_[l], d_[l]]

    mv(points = points, start_points = true, input_file = input_file, output_file = output_file, double_penetration = true)
    matrix = readdlm(output_file * ".csv", ',')
    # indexing from 2 because c
    coord_ab = matrix[2:end,4]
    
    matrixx = readdlm(output_file * "double" * ".csv", ',')
    coord_cd = matrixx[2:end,4]

    a_0 = points[1]
    b_0 = points[2]
    d_0 = points[4]
    c_0 = points[3]

    vec_0 = Array([data[a_0][1],data[a_0][2],data[a_0][3]])
    vec_1 = Array([data[b_0][1],data[b_0][2],data[b_0][3]])
    vec_2 = Array([data[c_0][1],data[c_0][2],data[c_0][3]])
    vec_3 = Array([data[d_0][1],data[d_0][2],data[d_0][3]])
    
    
    a_0_set, dists = knn(kdtree, vec_0,K, true)
    b_0_set, dists = knn(kdtree, vec_1,K, true)
    c_0_set, dists = knn(kdtree, vec_2,K, true)
    d_0_set, dists = knn(kdtree, vec_3,K, true)
    


    # find b_0^i and d_0^i that minimizes loss
    a_0_i = a_0
    b_0_i = b_0
    c_0_i = c_0
    d_0_i = d_0

    # See if stationary
    old_a = a_0
    old_b = b_0
    old_c = c_0
    old_d = d_0

    if true#mod(k,2) == 1

        for i in 1:length(b_0_set)
            b_0_i = b_0_set[i]
            for j in 1:length(c_0_set)
                c_0_i  = c_0_set[j]
                for kk in 1:length(d_0_set)
                    d_0_i  = d_0_set[kk]
                    #l = loss([a_0,b_0_i,c_0_i,d_0_i], coord_ab, coord_cd)
                    l = loss([a_0,b_0_i,c_0_i,d_0_i], coord_ab, coord_cd)
                    if  l < min_loss
                        min_loss = l
                        b_0 = b_0_i
                        c_0_ = c_0_i
                        # Let's freeze one variable
                        d_0 = d_0_i 
                    end
                end
            end
        end


        """
        for i in 1:length(b_0_set)
            b_0_i = b_0_set[i]
            l = loss([a_0,b_0_i,c_0,d_0], coord_ab, coord_cd)
            if  l < min_loss
                #min_loss = l
                b_0 = b_0_i
                # Let's freeze one variable
                #d_0 = d_0_i 
            end
        end
        
        for j in 1:length(d_0_set)
            d_0_i  = d_0_set[j]
            l = loss([a_0,b_0,c_0,d_0_i], coord_ab, coord_cd)
            if l < min_loss
                #min_loss = l
                
                # Let's freeze one variable
                d_0 = d_0_i 
            end
        end
        

        # FIX a_0 allways
        for j in 1:length(c_0_set)
            c_0_i  = c_0_set[j]
            l = loss([a_0,b_0,c_0_i,d_0], coord_ab, coord_cd)
            if l < min_loss
                min_loss = l
                # Let's freeze one variable
                c_0 = c_0_i 
            end
        end
        

        end

    
    # Repeat same thing but the other variable
    for i in 1:length(b_0_set)
        b_0_i = b_0_set[i]
        for j in 1:length(d_0_set)
            d_0_i  = d_0_set[j]
            if loss([a_0,b_0_i,c_0,d_0_i], coord_ab, coord_cd) <= min_loss
                min_loss = loss([a_0,b_0_i,c_0,d_0_i], coord_ab, coord_cd)
                #b_0 = b_0_i
                # Let's freeze one variable
                d_0 = d_0_i 
            end
        end
    end
    """
    end

    if old_b == b_0 && old_d == d_0 && old_a == a_0 && old_c == c_0
        break_index -=1
        println("THIS IS A FIXPOINT")
        
        #K += 10
        #println("K = ", K)
    else
        break_index = KKK
    end

    println("")
    println("Loss: ", min_loss)
    push!(loss_, min_loss)
    #println("Points vector: ", points)
    #println("Points: ", a_0,"-", b_0,"-", c_0,"-", d_0)
    println("")
    push!(a_, a_0)
    push!(b_, b_0)
    push!(c_, c_0)
    push!(d_, d_0)

    if break_index == 0
        break
    end
    println("________________________________________________________________")
    println("Done with: ", k, " Break_index: ", break_index)
    println("________________________________________________________________")


end

In [ ]:
loss_

In [ ]:
println("Input:")

println("[" * string(a_[1]-1) * "," * string(b_[1]-1) * "," * string(c_[1]-1) * "," * string(d_[1]-1) * "]")

In [ ]:
l = length(a_)
println("Output:")
print("[" * string(a_[l]-1) * "," * string(b_[l]-1) * "," * string(c_[l]-1) * "," * string(d_[l]-1) * "]")